É uma sequência de camadas e podemos usar o modelo sequencial oferecido pelo Keras, que possui as funções necessárias para construir cada camada de uma rede neural convolucional

O Keras utiliza o TensorFlow como backend, pois, na prática, O Keras é apenas uma biblioteca para simplificar a complexidade do TensorFlow. Aqui estão as versões utilizadas:

In [2]:
#Imports
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

Inicializando a rede neural convolucional

In [12]:
classifier = Sequential()

Defininco parâmetros para o shape dos dados de entrada e a função de ativação. Usaremos 32 features para um array 2D e definiremos nosso array como o formato 3x3

Converteremos todas as nossas imagens 64x64 pixels em um array 3D (pois as imagens são coloridas com 3 canais de cores)

In [13]:
#Passo 1 - Primeira Camada de Convolução
classifier.add(
    Conv2D(32, (3,3),input_shape = 
           (64,64,3), activation = 'relu'))

Em seguida, aplicamos o agrupamento (pooling) para reduzir o tamanho do mapa de features, resultado da primeria camada de convolução (dividido por 2):

In [14]:
#Passo 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

Adicionamos, então, a segunda camada de convolução, tornando nossa rede um pouco mais profunda:

In [15]:
#Adicionando a Segunda Camada de Convolução
classifier.add(Conv2D(32, (3,3), 
                      activation = 'relu'))

Mais uma vez, aplicamos a camada de pooling à saída da camada de convolução anterior

In [16]:
classifier.add(MaxPooling2D(pool_size = (2,2)))

Agora, aplicamos o "achatamento", ou apenas Flatten, para converter a estrutura de dados 2D, resultando da camada anterior em uma estrutura 1D, ou seja, um vetor

In [17]:
#Passo 3 - Flattening
classifier.add(Flatten())

No próximo passo, conectamos todas as camadas. Usamos uma função de ativação retificadora (relu) e então uma função de ativação sigmoide, para obter as probabilidados de cada imagem conter um cachorro ou um gato. O modelo raramente terá 100% de certeza e o que ele gera como resultado é uma probabilidade.

In [18]:
#Passo 4 - Full connection 
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

Finalmente, compilamos nossa rede neural. Para compilar a rede, usamos o otimizador "Adam", um excelente algoritmo para otimização baseada em gradiente de funções objetivas, estocásticas, que toma como base uma estimativa adaptada de momneto de baixa ordem.

Usamos uma função log loss com "entropia binária cruzada", pois ela funciona bem com funções sigmoides. Nossa métrica será a acurácia, pois essa é nossa maior preocupação no treinamento desse tipo de modelo.

In [19]:
#Compilando a rede
classifier.compile(optimizer = 'adam', 
                   loss = 'binary_crossen_tropy',
                   metrics = ['accuracy'])

Nesse ponto, temos nossa rede contruída. Precisamos, agora, treiná-la

### Treinando a rede neural convolucional

### Pré-processamento

Para essa tarefa, vamos usar a função "ImageDataGenerator()" do Keras e ajustar escala e zoom das imagens de treino e a escala das imagens de validação.

In [21]:
#Criando os objetos train_datagen e 
#validation_datagen com as regras de 
#pré-processamento das imagens

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

validation_datagen = ImageDataGenerator(rescale = 1./255)

Aplicamos, então, os dois objetos criados anteriormente para pré-processar os dados de treino e validação. Lembre-se: o tratamento aplicado aos dados de validação deve ser o mesmo aplicado aos dados de treino.

In [22]:
#Pré-processamento das imagens de treino e validação
training_set = train_datagen.flow_from_directory('dogs-vs-cats/dataset_treino/',
                                                 target_size=(64,64),
                                                 batch_size=32,
                                                 class_mode='binary'
                                                 )
validation_set = validation_datagen.flow_from_directory('dogs-vs-cats/dataset_validation/',
                                                        target_size=(64,64),
                                                        batch_size=32,
                                                        class_mode='binary'
                                                        )

Found 25000 images belonging to 2 classes.
Found 0 images belonging to 0 classes.


### Treinamento

Usaremos 8.000 passos em nosso conjunto de treinamento para cada época. Escolhemos 2.000 etapas de validação para as imagens de validação - esses hiperparâmetros são definidos por você

In [23]:
#Executando o treinamento
data_gen_train = ImageDataGenerator(rescale = 1/255.)
data_gen_valid = ImageDataGenerator(rescale = 1/255.)

train_generator = data_gen_train.flow_from_directory(
    'dogs-vs-cats/dataset_treino/',
    target_size = (64,64),
    batch_size = 32,
    class_mode = 'binary')

valid_generator = data_gen_valid.flow_from_directory(
    'dogs-vs-cats/dataset_validation/',
    target_size = (64,64),
    batch_size = 32,
    class_mode = 'binary')

classifier.fit(train_generator,
                         epochs = 2,
                         validation_data = valid_generator)

Found 25000 images belonging to 2 classes.
Found 0 images belonging to 0 classes.


2021-10-21 12:36:20.898124: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-10-21 12:36:20.958165: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3400365000 Hz


Epoch 1/2


ValueError: in user code:

    /home/gabriel/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /home/gabriel/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/gabriel/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/gabriel/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/gabriel/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/gabriel/anaconda3/lib/python3.8/site-packages/tensorf